# Merge All Data

Merge air quality, AEMET climate and ERA5-Land data for each station.

In [1]:
# Load packages
suppressMessages(library(tidyverse))
suppressMessages(library(lubridate))

In [2]:
# Working directory
setwd("~/Repositories/AirQualityCOVID")

# Source some scripts 
source("src/Curation/functions.R")
source("src/functions.R")

### Main Variables

In [3]:
start_dt <- ymd_hms("2013-01-01 00:00:00")
end_dt <- ymd_hms("2020-12-31 00:00:00")

## Stations Information

In [4]:
#-----------------------------
#    Air Quality Stations
#-----------------------------

sitesAQ <- read.csv("data/Curation/AirQuality/checked_sitesAQ.csv",
                    stringsAsFactor=F)

#-----------------------------
#       AEMET Stations
#-----------------------------

sitesMto <- read.csv("data/Curation/AEMET/checked_sites_AEMET.csv",
                     stringsAsFactor=F)

In [5]:
#--------------------------------
#       Relevant Information
#--------------------------------

#sites.lv <- levels(as.factor(sitesAQ$site))[1:2]
#
# Madrid, Santander and Barcelona stations
sites.lv <- c("es0118a", "es1580a", "es1480a")

In [6]:
#-----------------------------
#    ERA5-Land Data
#-----------------------------

folder.ERA5.Land <- "data/Curation/ERA5-Land/Values/"

# Relative Humidity
rh.ERA5.Land <- read.csv(paste(folder.ERA5.Land,
                               "rh_daily_2010_2020_final_stations.csv",
                               sep=""), stringsAsFactor=F) %>% 
                    data.as.datetime("dates", "ymd") %>%
                    add_column(variable="RH", .after="dates")

# Solar radiation
ssrd.ERA5.Land <- read.csv(paste(folder.ERA5.Land,
                                 "ssrd_daily_2010_2020_final_stations.csv",
                                 sep=""),stringsAsFactor=F) %>% 
                    data.as.datetime("dates", "ymd") %>%
                    add_column(variable="solar.radiation", .after="dates")

ERA5.Land <- rbind(rh.ERA5.Land, ssrd.ERA5.Land)

## Merge Process

In [7]:
# Folder wherever take downloaded data, if it exists
AQ.files <- "data/Curation/AirQuality/Values/"
Mto.files <- "data/Curation/AEMET/Values/"

In [11]:
list.data <- list()

for (st in sites.lv) {
    
    #-----------------------------
    #    Air Quality Data
    #-----------------------------
    
    dataAQ <- get.AQdata(st, 
                         sitesAQ[sitesAQ$site == st, "Pollutant"], 
                         start_dt=start_dt, end_dt=end_dt,
                         data.by.file=TRUE, fileName=AQ.files) %>%
                    group.by.date(formulation = value ~ date + site + variable,
                                  dateCl="date", unit="day", FUN="mean") %>%
                    data.as.datetime("date", "ymd")%>%
                    pivot.long.table(valueCl = "value", variableCl="variable") %>%
                    select(-site)
    
    #----------------------------------
    #       Meteorological Data
    #----------------------------------
    
    indicativo <- sitesMto[sitesMto$siteAQ == st, "indicativo"][1]
    print(indicativo)
    
    dataMto <- read.csv(paste(Mto.files,
                              indicativo, ".csv",
                              sep=""), stringsAsFactor=F) %>%
                    data.as.datetime("fecha", "ymd") %>%
                    select("fecha", "tmed", "prec", "tmin", "tmax", "dir",
                           "velmedia", "racha", "presMax", "presMin"
                           )
    
    for (vr in levels(as.factor(ERA5.Land$variable))) {
        merge.data <- merge(x = dataMto, 
                            y = ERA5.Land[ERA5.Land$variable == vr,
                                          c("dates", st)],
                            by.x = "fecha", by.y = "dates", all.x = TRUE
                           )
        names(merge.data)[ncol(merge.data)] <- vr
    }    
    
    list.data[[st]] <- list(dataAQ=dataAQ,
                            dataMto=dataMto,
                            indicativo=indicativo
                           )
}

[1] "3195"
[1] "1111"
[1] "0200E"


## Guardar Datos en csv 

```R

save(list.data, sites.lv, "data/Curation/values.rda")

```